In [55]:
import folium
import numpy as np
import pandas as pd
# from js import fetch
import io
import ssl

In [56]:
#San Francisco incidents map
url =  'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/Police_Department_Incidents_-_Previous_Year__2016_.csv'
ssl._create_default_https_context = ssl._create_unverified_context
df_incidents = pd.read_csv(url)
# df_incidents

In [64]:
limit = 180
df_incidents = df_incidents.iloc[0:limit, :]

In [58]:
# San Francisco latitude and longitude values
latitude = 37.77
longitude = -122.42

In [60]:
sanfran_map = folium.Map(location=[latitude, longitude], zoom_start=12)

sanfran_map

In [61]:
incidents = folium.map.FeatureGroup()


for lat, lng, in zip(df_incidents.Y, df_incidents.X):
    incidents.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, 
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

sanfran_map.add_child(incidents)

In [62]:
sanfran_map = folium.Map(location=[latitude, longitude], zoom_start=12)


for lat, lng, label in zip(df_incidents.Y, df_incidents.X, df_incidents.Category):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5, 
        color='yellow',
        fill=True,
        popup=label,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(sanfran_map)

# show map
sanfran_map

In [65]:
from folium import plugins


sanfran_map = folium.Map(location = [latitude, longitude], zoom_start = 12)


incidents = plugins.MarkerCluster().add_to(sanfran_map)


for lat, lng, label, in zip(df_incidents.Y, df_incidents.X, df_incidents.Category):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(incidents)

sanfran_map

In [76]:
# Canadian Immigration map
canadianImmigration_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/Canada.xlsx'

ssl._create_default_https_context = ssl._create_unverified_context
df_can_imm = pd.read_excel(canadianImmigration_url,
                          sheet_name='Canada by Citizenship',
                          skiprows=range(20),
                          skipfooter=2)
# df_can_imm

In [77]:
# clean up the dataset to remove unnecessary columns (eg. REG) 
df_can_imm.drop(['AREA','REG','DEV','Type','Coverage'], axis=1, inplace=True)

# let's rename the columns so that they make sense
df_can_imm.rename(columns={'OdName':'Country', 'AreaName':'Continent','RegName':'Region'}, inplace=True)

# for sake of consistency, let's also make all column labels of type string
df_can_imm.columns = list(map(str, df_can_imm.columns))

# add total column
df_can_imm['Total'] = df_can_imm.sum(axis=1)

# years that we will be using in this lesson - useful for plotting later on
years = list(map(str, range(1980, 2014)))
print ('data dimensions:', df_can_imm.shape)

data dimensions: (195, 39)


/var/folders/y4/1hq_lmfj72zdtzqv5xmsswxr0000gn/T/ipykernel_63777/492616641.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_can_imm['Total'] = df_can_imm.sum(axis=1)


In [78]:
print(df_can_imm.shape)

(195, 39)


In [88]:
import requests
import json

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/world_countries.json'
response = requests.get(URL)
data = response.json()
world_geo = data

In [92]:
# create a plain world map
world_map = folium.Map(location=[0, 0], zoom_start=1)

In [95]:
# create a numpy array of length 6 and has linear spacing from the minimum total immigration to the maximum total immigration
threshold_scale = np.linspace(df_can_imm['Total'].min(),
                              df_can_imm['Total'].max(),
                              6, dtype=int)
threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[-1] = threshold_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum immigration

# let Folium determine the scale.
world_map = folium.Map(location=[0, 0], zoom_start=2)
world_map.choropleth(
    geo_data=world_geo,
    data=df_can_imm,
    columns=['Country', 'Total'],
    key_on='feature.properties.name',
    threshold_scale=threshold_scale,
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Immigration to Canada',
    reset=True
)
world_map